In [ ]:
import uxarray as ux

In [ ]:
uxgrid = ux.open_grid("../../test/meshfiles/geos-cs/c12/test-c12.native.nc4")

In [ ]:
merged_grid = uxgrid.merge_duplicate_node_indices()

In [ ]:
new_grid = merged_grid.compute_dual()

In [ ]:
new_grid.plot(backend="matplotlib", fig_size=200) + uxgrid.plot(
    backend="matplotlib", fig_size=200, color="red"
)